In [1]:
from core.data import from_dir, random_resample

data_path = 'data/chest_xray/train'

In [2]:
resize = (96, 96)

In [3]:
training_selector = lambda s: s.startswith('chest_xray/train/') and s.endswith('.jpeg')
(x, y) = from_dir(data_path, resize=resize, resample=random_resample)

In [4]:
NEG_IDX = 0
POS_IDX = 1
TOTAL   = len(y)
N       = len(list(filter(lambda j: j == 0, y)))
P       = len(list(filter(lambda j: j == 1, y)))
print(f'#Positives = {P} ({P/TOTAL*100:.2f}%)')
print(f'#Negatives = {N} ({N/TOTAL*100:.2f}%)')

#Positives = 3883 (74.22%)
#Negatives = 1349 (25.78%)


In [5]:
from sklearn.model_selection import train_test_split

X_train, X_val, Y_train, Y_val = train_test_split(x, y, test_size=0.2, random_state=42)
total_train = len(Y_train)
N_train     = len(list(filter(lambda j: j == 0, Y_train)))
P_train     = len(list(filter(lambda j: j == 1, Y_train)))
print(f'#Positives (train) = {P_train} ({P_train/total_train*100:.2f}%)')
print(f'#Negatives (train) = {N_train} ({N_train/total_train*100:.2f}%)')
total_val   = len(Y_val)
N_val       = len(list(filter(lambda j: j == 0, Y_val)))
P_val       = len(list(filter(lambda j: j == 1, Y_val)))
print(f'#Positives (val)   = {P_val} ({P_val/total_val*100:.2f}%)')
print(f'#Negatives (val)   = {N_val} ({N_val/total_val*100:.2f}%)')

#Positives (train) = 3095 (73.95%)
#Negatives (train) = 1090 (26.05%)
#Positives (val)   = 788 (75.26%)
#Negatives (val)   = 259 (24.74%)


In [9]:
from core import preprocessing
import numpy as np

def apply_cutmix(x, y):
    shf = list(range(len(x)))
    np.random.shuffle(shf)
    x_mix = []
    y_mix = []
    for i in range(len(x)):
        a = (x[i], y[i])
        b = (x[shf[i]], y[shf[i]])
        xc, yc = preprocessing.cutmix(a, b)
        x_mix.append(xc)
        y_mix.append(yc)
    return np.asarray(x_mix), np.asarray(y_mix)

In [6]:
from keras import callbacks as cb, losses, metrics as metr, optimizers
from core import create_model, preprocessing

callbacks = [cb.EarlyStopping(patience=5, verbose=1)]
model = create_model('resnet-18', resize)
metrics = [metr.BinaryCrossentropy(name='loss'), metr.BinaryCrossentropy('val_loss'), metr.Recall(name='recall'), metr.Recall(name='val_recall')]
optimizer = optimizers.Adam()
loss = losses.BinaryCrossentropy()

2025-02-23 22:29:00.889498: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-23 22:29:00.889813: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 22:29:00.891402: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-23 22:29:00.894770: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740346140.900097  123090 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740346140.90

In [10]:
from core import loop


history = loop.training_loop(
    X_train, Y_train, X_val, Y_val,
    model,
    4,
    64,
    metrics,
    optimizer,
    callbacks,
    loss,
    apply_cutmix)
history

Epoch 1/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 1:02 987ms/stepTraining loss (for one batch) at step 0: 0.6816
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/steploss: 0.37567073106765747,recall: 0.830373227596283
val_loss: 0.14289772510528564,val_recall: 0.9911167621612549
64/64 ━━━━━━━━━━━━━━━━━━━━ 14s 206ms/step - loss: 0.3757 - val_loss: 0.1429
Epoch 2/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 11s 177ms/stepTraining loss (for one batch) at step 0: 0.3015
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/steploss: 0.3249145448207855,recall: 0.822073221206665
val_loss: 0.23028160631656647,val_recall: 0.9568527936935425
64/64 ━━━━━━━━━━━━━━━━━━━━ 12s 194ms/step - loss: 0.3249 - val_loss: 0.2303
Epoch 3/4
 1/64 ━━━━━━━━━━━━━━━━━━━━ 11s 190ms/stepTraining loss (for one batch) at step 0: 0.3632
Seen so far: 64 samples
66/64 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/steploss: 0.32040974497795105,recall: 0.8268387317657471
val_loss: 0.16413483023643494,val_recall: 0.9949238300323486
64/64 ━━━━━━━━━━━━━